## 예술의 전당 

In [1]:
import pandas as pd
aug_df = pd.read_csv(r'..\preprocessed_data\예술의 전당\예술의전당_02_22년.csv', index_col = 0, encoding = 'cp949')
display(aug_df)

C:\Users\passion\AppData\Local\Temp\ipykernel_25524\740946338.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  aug_df = pd.read_csv(r'..\preprocessed_data\예술의 전당\예술의전당_02_22년.csv', index_col = 0, encoding = 'cp949')


,전송사업자명,좌석수,공연코드,공연회차,공연일시,예매/취소구분,예매/취소매수,예매/취소금액,장당금액,공연시작일자,...,극작가명,수상실적,관람연령,아동공연 여부,축제 여부,내한공연 여부,오픈런 여부,좌석등급,단독판매여부,판매좌석수
전송사업자코드,,,,,,,,,,,,,,,,,,,,,
SE000101,예스24,354,PF306909,1,2022-01-09 14:00,1,1,10000,20000,2022-01-09,...,NaN,NaN,만 7세 이상,N,N,N,N,전석(20000),N,0석
SE000101,예스24,354,PF306909,1,2022-01-09 14:00,1,1,10000,20000,2022-01-09,...,NaN,NaN,만 7세 이상,N,N,N,N,전석(20000),N,0석
SE000101,예스24,354,PF306909,1,2022-01-09 14:00,1,1,10000,20000,2022-01-09,...,NaN,NaN,만 7세 이상,N,N,N,N,전석(20000),N,0석
SE000101,예스24,354,PF306909,1,2022-01-09 14:00,1,1,10000,20000,2022-01-09,...,NaN,NaN,만 7세 이상,N,N,N,N,전석(20000),N,0석
SE000101,예스24,354,PF306909,1,2022-01-09 14:00,1,1,10000,20000,2022-01-09,...,NaN,NaN,만 7세 이상,N,N,N,N,전석(20000),N,0석
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SE000201,인터파크,354,PF425231,1,2022-12-31 19:30,1,1,20000,20000,2022-12-31,...,NaN,NaN,만 7세 이상,N,N,N,N,전석(20000),N,354석
SE000201,인터파크,354,PF425231,1,2022-12-31 19:30,1,1,20000,20000,2022-12-31,...,NaN,NaN,만 7세 이상,N,N,N,N,전석(20000),N,354석
SE000201,인터파크,354,PF425231,1,2022-12-31 19:30,1,1,20000,20000,2022-12-31,...,NaN,NaN,만 7세 이상,N,N,N,N,전석(20000),N,354석


In [104]:
import pandas as pd
import numpy as np
import re
from scipy import stats

aug_df['공연일시'] = pd.to_datetime(aug_df['공연일시'])

aug_df['공연시작년도'] = aug_df['공연일시'].dt.year
aug_df['공연시작월'] = aug_df['공연일시'].dt.month
aug_df['공연시작일'] = aug_df['공연일시'].dt.day
aug_df['공연시작시간'] = aug_df['공연일시'].dt.hour
aug_df['공연시작요일'] = aug_df['공연일시'].dt.day_name()
aug_df['공연시작분'] = aug_df['공연일시'].dt.minute

aug_df['공연시작시분']=aug_df['공연시작시간']*60 + aug_df['공연시작분']
aug_df.drop(['공연시작분'], axis=1, inplace = True)

aug_df['공연시작일자'] = pd.to_datetime(aug_df['공연시작일자'])
aug_df['공연종료일자'] = pd.to_datetime(aug_df['공연종료일자'])

aug_df['공연기간'] = (aug_df['공연시작일자'] - aug_df['공연시작일자']).dt.days + 1
aug_df = aug_df.drop(['공연시작일자','공연종료일자'], axis=1)

#공휴일 여부
public_holidays = ['2019-07-17', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25', 
                   '2020-01-01', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-03-01', '2020-04-15', '2020-04-30', 
                   '2020-05-05', '2020-06-06', '2020-08-15', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', 
                   '2020-12-25', '2021-01-01', '2021-02-11', '2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', 
                   '2021-06-06', '2021-08-15', '2021-08-16', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', 
                   '2021-10-11', '2021-12-25', '2022-01-01', '2022-01-31', '2022-02-01', '2022-02-02', '2022-03-01', '2022-03-09', 
                   '2022-05-05', '2022-05-08', '2022-06-01', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', 
                   '2022-10-03', '2022-10-09', '2022-10-09', '2022-12-25']

def is_public_holiday_code(date):
    return 1 if str(date.date()) in public_holidays else 0

aug_df['공휴일여부'] = aug_df['공연일시'].apply(is_public_holiday_code)


def num_of_class(text):
    if pd.isna(text):
        result = 1
    else:
        li = []
        for i in text.split():
            li.append(int(re.sub(r'[^0-9]', '', i)))
        result = len(li)
    return result

def avg_price(text):
    if pd.isna(text):
        result = 0
    else:
        li = []
        for i in text.split():
            li.append(int(re.sub(r'[^0-9]', '', i)))
        result = np.mean(li)
    return result


aug_df['좌석등급개수'] = aug_df['좌석등급'].apply(num_of_class)
aug_df['평균티켓가격'] = aug_df['좌석등급'].apply(avg_price)


#좌석점유율
dict_frac = dict()

def extract_number(text):
    if pd.isna(text):
        return 0
    else:
        # 숫자와 관련 없는 문자 제거 후 숫자만 추출
        numbers = ''.join(filter(str.isdigit, text))
        return int(numbers)

dict_frac = dict()

for p in aug_df['공연코드'].unique():
    print(p)
    sub_df = aug_df[aug_df['공연코드']==p]
    saled = max([extract_number(text) for text in sub_df['판매좌석수'].unique()])
    if saled == 0:
        dict_frac[p] = saled
        print(saled)
    if len(np.unique(sub_df['예매/취소구분'], return_counts = True)[0]) == 1:
        print(1)
        book = np.unique(sub_df['예매/취소구분'], return_counts = True)[1][0]
        dict_frac[p] = book/saled
        print(book/saled)
    elif len(np.unique(sub_df['예매/취소구분'], return_counts = True)[0]) == 2:
        print(2)
        book, cancel = np.unique(sub_df['예매/취소구분'], return_counts = True)[1]
        dict_frac[p]= (book-cancel)/saled
        print((book-cancel)/saled)
    else:
        print('error')
   
aug_df['좌석점유율'] = aug_df['공연코드'].apply(lambda x: dict_frac[x])


#소요시간(분)
def convert_time_to_minutes(time_string):
    hours = 0
    minutes = 0
    if '시간' in time_string:
        hours = int(time_string.split('시간')[0])
        # '분'이 추가적으로 있는 경우, '시간' 뒤에 오는 문자열에서 추출합니다.
        if '분' in time_string.split('시간')[1]:
            minutes = int(time_string.split('시간')[1].split('분')[0])
    elif '분' in time_string:
        minutes = int(time_string.split('분')[0])
    return hours*60 + minutes

aug_df['소요시간'] = aug_df['소요시간'].apply(convert_time_to_minutes)

aug_df.drop(['전송사업자명','좌석수','공연일시','예매/취소매수','예매/취소구분','판매좌석수','예매/취소금액','장당금액','원작자명',
         '극작가명','좌석등급','판매좌석수','공연시작시간','수상실적'],axis = 1, inplace = True)

aug_df.drop_duplicates(inplace=True)

PF306909
2
0.6468926553672316
PF306012
2
0.6412429378531074
PF307300
2
0.6214689265536724
PF306155
2
0.6836158192090396
PF307487
2
0.4604519774011299
PF307439
2
0.4463276836158192
PF308278
1
0.4152542372881356
PF307474
2
0.4971751412429379
PF307385
2
0.6242937853107344
PF307004
2
0.6045197740112994
PF307810
2
0.5536723163841808
PF307429
2
0.4915254237288136
PF308589
1
0.00847457627118644
PF307493
2
0.4689265536723164
PF307210
2
0.5819209039548022
PF307892
2
0.5480225988700564
PF307941
2
0.4689265536723164
PF308462
2
0.4971751412429379
PF307500
2
0.5169491525423728
PF307743
2
0.4943502824858757
PF308700
1
0.2740112994350282
PF307918
2
0.635593220338983
PF308541
2
0.6440677966101694
PF308400
2
0.615819209039548
PF307900
2
0.6412429378531074
PF307906
2
0.4717514124293785
PF307099
2
0.5423728813559322
PF307147
2
0.6610169491525424
PF308457
2
0.5
PF307935
2
0.4717514124293785
PF307100
2
0.632768361581921
PF307503
2
0.5988700564971752
PF307566
2
0.4576271186440678
PF308785
2
0.50564971751412

2
0.9548022598870056
PF311071
2
0.963276836158192
PF422310
1
0.963276836158192
PF422313
2
0.384180790960452
PF423553
2
0.96045197740113
PF422099
2
0.4774011299435028
PF422889
2
0.4180790960451977
PF422829
2
0.9209039548022598
PF422815
2
0.9548022598870056
PF423380
2
0.9519774011299436
PF422698
2
0.9576271186440678
PF311879
2
0.9350282485875706
PF422741
2
0.9689265536723164
PF422736
2
0.9830508474576272
PF423731
2
0.8757062146892656
PF422027
2
0.9378531073446328
PF422903
2
0.9830508474576272
PF423294
2
0.9774011299435028
PF423847
2
0.5734463276836158
PF422508
2
0.7853107344632768
PF425589
2
0.5225988700564972
PF424644
1
0.980225988700565
PF423682
2
0.8248587570621468
PF423845
2
0.807909604519774
PF424648
2
0.9887005649717514
PF422187
2
0.9519774011299436
PF422606
2
0.9265536723163842
PF423769
2
1.8192090395480225
PF425383
2
1.8926553672316384
PF424029
2
1.9039548022598871
PF424584
2
0.8870056497175142
PF425290
2
1.1242937853107344
PF422192
2
0.9689265536723164
PF425283
2
0.4915254237288

In [119]:
real_aug = pd.concat([real_aug,aug_df], axis = 0)
display(real_aug)

,공연코드,공연회차,소요시간,장르명,세부장르명,출연진내용,제작진내용,기획제작사명,관람연령,아동공연 여부,...,공연시작년도,공연시작월,공연시작일,공연시작요일,공연시작시분,공연기간,공휴일여부,좌석등급개수,평균티켓가격,좌석점유율
SE000201,PF372853,1,110,서양음악(클래식),기악,"김주원, 쾅 하오 후앙",NaN,(주)마스트미디어(주최),만 7세 이상,N,...,2019,7,2,Tuesday,1200,1,0,1,30000.0,0.460452
SE000201,PF360668,1,100,서양음악(클래식),기악,"최유진, 이재완",NaN,예인예술기획(주최),만 7세 이상,N,...,2019,7,3,Wednesday,1200,1,0,1,20000.0,0.968927
SE000201,PF372854,1,100,서양음악(클래식),기악,김민욱,NaN,가온클래식(주최),만 7세 이상,N,...,2019,7,4,Thursday,1200,1,0,1,20000.0,0.742938
SE000201,PF360799,1,80,서양음악(클래식),기악,"김경희, 신미정, 장혜리, 안동호",NaN,세광아트(세광음악출판사)(주관),만 7세 이상,N,...,2019,7,5,Friday,1200,1,0,1,30000.0,0.946541
SE000201,PF372069,1,100,서양음악(클래식),기악,나윤아,NaN,마에스트로 컴퍼니(주최),만 7세 이상,N,...,2019,7,6,Saturday,1200,1,0,1,10000.0,0.593525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SE000201,PF424244,1,90,서양음악(클래식),기악,"유의정, 김정현, 윤효린",NaN,지클레프(주최),만 7세 이상,N,...,2022,12,28,Wednesday,1170,1,0,1,20000.0,0.946328
SE000101,PF426471,1,90,서양음악(클래식),기악,이주용,NaN,NaN,만 7세 이상,N,...,2022,12,29,Thursday,1170,1,0,1,30000.0,0.728261
SE000201,PF425379,1,90,서양음악(클래식),기악,"김길우, 정태민",NaN,지클레프(주최),만 7세 이상,N,...,2022,12,30,Friday,1170,1,0,1,20000.0,0.951977
SE000201,PF424637,1,90,서양음악(클래식),기악,박요셉,NaN,지클레프(주최),만 7세 이상,N,...,2022,12,31,Saturday,840,1,0,2,40000.0,0.954802


In [120]:
real_aug.to_csv('..\preprocessed_data\예술의 전당\예술의전당.csv')

## 대구콘서트하우스 

In [3]:
import pandas as pd
aug_df = pd.read_csv(r'..\preprocessed_data\대구콘서트하우스\대구콘서트하우스_02_22년.csv', index_col = 0, encoding = 'cp949')
display(aug_df)

,전송사업자명,좌석수,공연코드,공연회차,공연일시,예매/취소구분,예매/취소매수,예매/취소금액,장당금액,공연시작일자,...,극작가명,수상실적,관람연령,아동공연 여부,축제 여부,내한공연 여부,오픈런 여부,좌석등급,단독판매여부,판매좌석수
전송사업자코드,,,,,,,,,,,,,,,,,,,,,
SE000102,티켓링크,248,PF307781,1,2022-01-08 15:00,1,1,0,0,2022-01-08,...,NaN,NaN,만 8세 이상,N,N,N,N,전석무료(0),N,1석
SE000102,티켓링크,248,PF307781,1,2022-01-08 15:00,1,1,0,0,2022-01-08,...,NaN,NaN,만 8세 이상,N,N,N,N,전석무료(0),N,1석
SE000102,티켓링크,248,PF307781,1,2022-01-08 15:00,1,1,0,0,2022-01-08,...,NaN,NaN,만 8세 이상,N,N,N,N,전석무료(0),N,1석
SE000102,티켓링크,248,PF307781,1,2022-01-08 15:00,1,1,0,0,2022-01-08,...,NaN,NaN,만 8세 이상,N,N,N,N,전석무료(0),N,1석
SE000102,티켓링크,248,PF307781,1,2022-01-08 15:00,1,1,0,0,2022-01-08,...,NaN,NaN,만 8세 이상,N,N,N,N,전석무료(0),N,1석
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SE000102,티켓링크,248,PF426422,1,2022-12-16 19:30,1,1,0,0,2022-12-16,...,NaN,NaN,만 5세 이상,N,N,N,N,전석(10000),Y,1석
SE000102,티켓링크,248,PF426422,1,2022-12-16 19:30,1,1,0,0,2022-12-16,...,NaN,NaN,만 5세 이상,N,N,N,N,전석(10000),Y,1석
SE000102,티켓링크,248,PF426422,1,2022-12-16 19:30,1,1,0,0,2022-12-16,...,NaN,NaN,만 5세 이상,N,N,N,N,전석(10000),Y,1석


In [13]:
import pandas as pd
import numpy as np
import re
from scipy import stats

aug_df['공연일시'] = pd.to_datetime(aug_df['공연일시'])

aug_df['공연시작년도'] = aug_df['공연일시'].dt.year
aug_df['공연시작월'] = aug_df['공연일시'].dt.month
aug_df['공연시작일'] = aug_df['공연일시'].dt.day
aug_df['공연시작시간'] = aug_df['공연일시'].dt.hour
aug_df['공연시작요일'] = aug_df['공연일시'].dt.day_name()
aug_df['공연시작분'] = aug_df['공연일시'].dt.minute

aug_df['공연시작시분']=aug_df['공연시작시간']*60 + aug_df['공연시작분']
aug_df.drop(['공연시작분'], axis=1, inplace = True)

aug_df['공연시작일자'] = pd.to_datetime(aug_df['공연시작일자'])
aug_df['공연종료일자'] = pd.to_datetime(aug_df['공연종료일자'])

aug_df['공연기간'] = (aug_df['공연시작일자'] - aug_df['공연시작일자']).dt.days + 1
aug_df = aug_df.drop(['공연시작일자','공연종료일자'], axis=1)

#공휴일 여부
public_holidays = ['2019-07-17', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25', 
                   '2020-01-01', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-03-01', '2020-04-15', '2020-04-30', 
                   '2020-05-05', '2020-06-06', '2020-08-15', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', 
                   '2020-12-25', '2021-01-01', '2021-02-11', '2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', 
                   '2021-06-06', '2021-08-15', '2021-08-16', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', 
                   '2021-10-11', '2021-12-25', '2022-01-01', '2022-01-31', '2022-02-01', '2022-02-02', '2022-03-01', '2022-03-09', 
                   '2022-05-05', '2022-05-08', '2022-06-01', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', 
                   '2022-10-03', '2022-10-09', '2022-10-09', '2022-12-25']

def is_public_holiday_code(date):
    return 1 if str(date.date()) in public_holidays else 0

aug_df['공휴일여부'] = aug_df['공연일시'].apply(is_public_holiday_code)


def num_of_class(text):
    if pd.isna(text):
        result = 1
    else:
        li = []
        for i in text.split():
            li.append(int(re.sub(r'[^0-9]', '', i)))
        result = len(li)
    return result

def avg_price(text):
    if pd.isna(text):
        result = 0
    else:
        li = []
        for i in text.split():
            li.append(int(re.sub(r'[^0-9]', '', i)))
        result = np.mean(li)
    return result


aug_df['좌석등급개수'] = aug_df['좌석등급'].apply(num_of_class)
aug_df['평균티켓가격'] = aug_df['좌석등급'].apply(avg_price)code = ['PF316358', 'PF317470','PF311804']

for j in code:
    sub_df = aug_df[aug_df['공연코드'] == j]
    for d in sub_df['공연일시'].unique():
        ssub_df = sub_df[sub_df['공연일시'] == d]
        if ssub_df['공연시작월'].values[0] < 4:
            max_sit = 117
        elif ssub_df['공연시작월'].values[0] == 4 and sub_df['공연시작일'].values[0] < 15:
            max_sit = 117
        elif ssub_df['공연시작월'].values[0] == 4 and sub_df['공연시작일'].values[0] >= 15:
            max_sit = 248
        elif ssub_df['공연시작월'].values[0] >= 5:
            max_sit = 248
        else:
            print('error')
        
        saled = max([extract_number(text) for text in ssub_df['판매좌석수'].unique()])
        if saled == 0 or saled == 1:
            if len(np.unique(ssub_df['예매/취소구분'], return_counts = True)[0]) == 1:
                book = np.unique(ssub_df['예매/취소구분'], return_counts = True)[1][0]
                aug_df.loc[np.logical_and(aug_df['공연코드'] == j, aug_df['공연일시'] == d),'좌석점유율'] = book/117
                print(book/max_sit)

            elif len(np.unique(ssub_df['예매/취소구분'], return_counts = True)[0]) == 2:
                book, cancel = np.unique(ssub_df['예매/취소구분'], return_counts = True)[1]
                aug_df.loc[np.logical_and(aug_df['공연코드'] == j, aug_df['공연일시'] == d),'좌석점유율'] = (book-cancel)/117
                print((book-cancel)/max_sit)

        elif len(np.unique(ssub_df['예매/취소구분'], return_counts = True)[0]) == 1:
            book = np.unique(ssub_df['예매/취소구분'], return_counts = True)[1][0]
            aug_df.loc[np.logical_and(aug_df['공연코드'] == j, aug_df['공연일시'] == d),'좌석점유율'] = book/saled
            print(book/saled)
        elif len(np.unique(ssub_df['예매/취소구분'], return_counts = True)[0]) == 2:
            book, cancel = np.unique(ssub_df['예매/취소구분'], return_counts = True)[1]
            aug_df.loc[np.logical_and(aug_df['공연코드'] == j, aug_df['공연일시'] == d),'좌석점유율'] = (book-cancel)/saled
            print((book-cancel)/saled)
        else:
            print('error')





def extract_number(text):
    if pd.isna(text):
        return 0
    else:
        # 숫자와 관련 없는 문자 제거 후 숫자만 추출
        numbers = ''.join(filter(str.isdigit, text))
        return int(numbers)

dict_frac = dict()

for p in aug_df['공연코드'].unique():
    print(p)
    sub_df = aug_df[aug_df['공연코드']==p]
    if sub_df['공연시작월'].values[0] < 4:
        max_sit = 117
    elif sub_df['공연시작월'].values[0] == 4 and sub_df['공연시작일'].values[0] < 15:
        max_sit = 117
    elif sub_df['공연시작월'].values[0] == 4 and sub_df['공연시작일'].values[0] >= 15:
        max_sit = 248
    elif sub_df['공연시작월'].values[0] >= 5:
        max_sit = 248
    else:
        print('error')
    print(max_sit)
    if len(sub_df['공연일시'].unique()) > 1:
        print('multiple date')
        print(len(sub_df['공연일시'].unique()))
    saled = max([extract_number(text) for text in sub_df['판매좌석수'].unique()])
    if saled == 0 or saled == 1:
        if len(np.unique(sub_df['예매/취소구분'], return_counts = True)[0]) == 1:
            book = np.unique(sub_df['예매/취소구분'], return_counts = True)[1][0]
            dict_frac[p] = book/max_sit
            print(book/max_sit)
            
        elif len(np.unique(sub_df['예매/취소구분'], return_counts = True)[0]) == 2:
            book, cancel = np.unique(sub_df['예매/취소구분'], return_counts = True)[1]
            dict_frac[p]= (book-cancel)/max_sit
            print((book-cancel)/max_sit)
            
    elif len(np.unique(sub_df['예매/취소구분'], return_counts = True)[0]) == 1:
        book = np.unique(sub_df['예매/취소구분'], return_counts = True)[1][0]
        dict_frac[p] = book/saled
        print(book/saled)
    elif len(np.unique(sub_df['예매/취소구분'], return_counts = True)[0]) == 2:
        book, cancel = np.unique(sub_df['예매/취소구분'], return_counts = True)[1]
        dict_frac[p]= (book-cancel)/saled
        print((book-cancel)/saled)
    else:
        print('error')
   
aug_df['좌석점유율'] = aug_df['공연코드'].apply(lambda x: dict_frac[x])

def convert_time_to_minutes(time_string):
    hours = 0
    minutes = 0
    if '시간' in time_string:
        hours = int(time_string.split('시간')[0])
        # '분'이 추가적으로 있는 경우, '시간' 뒤에 오는 문자열에서 추출합니다.
        if '분' in time_string.split('시간')[1]:
            minutes = int(time_string.split('시간')[1].split('분')[0])
    elif '분' in time_string:
        minutes = int(time_string.split('분')[0])
    return hours*60 + minutes

aug_df['소요시간'] = aug_df['소요시간'].apply(convert_time_to_minutes)

PF307781
117
0.9658119658119658
PF307583
117
0.9658119658119658
PF307308
117
0.9658119658119658
PF307824
117
0.9658119658119658
PF308256
117
0.9658119658119658
PF307107
117
0.9401709401709402
PF308790
117
0.9658119658119658
PF308788
117
0.9658119658119658
PF308109
117
0.9658119658119658
PF308016
117
0.9658119658119658
PF308749
117
0.9487179487179487
PF309443
117
0.9658119658119658
PF308033
117
0.9572649572649573
PF308372
117
0.05309734513274336
PF308141
117
0.07964601769911504
PF309804
117
0.9658119658119658
PF309213
117
0.9658119658119658
PF309842
117
0.9658119658119658
PF309086
117
0.9658119658119658
PF309349
117
0.9658119658119658
PF308998
117
0.9658119658119658
PF300485
117
0.9658119658119658
PF308147
117
0.9829059829059829
PF309481
117
1.0
PF309521
117
0.7948717948717948
PF309492
117
0.9572649572649573
PF309197
117
0.9658119658119658
PF301297
117
0.9658119658119658
PF300199
117
0.9658119658119658
PF301243
117
0.9658119658119658
PF309000
117
0.5470085470085471
PF300627
117
0.965811

In [14]:
code = ['PF316358', 'PF317470','PF311804']

for j in code:
    sub_df = aug_df[aug_df['공연코드'] == j]
    for d in sub_df['공연일시'].unique():
        ssub_df = sub_df[sub_df['공연일시'] == d]
        if ssub_df['공연시작월'].values[0] < 4:
            max_sit = 117
        elif ssub_df['공연시작월'].values[0] == 4 and sub_df['공연시작일'].values[0] < 15:
            max_sit = 117
        elif ssub_df['공연시작월'].values[0] == 4 and sub_df['공연시작일'].values[0] >= 15:
            max_sit = 248
        elif ssub_df['공연시작월'].values[0] >= 5:
            max_sit = 248
        else:
            print('error')
        
        saled = max([extract_number(text) for text in ssub_df['판매좌석수'].unique()])
        if saled == 0 or saled == 1:
            if len(np.unique(ssub_df['예매/취소구분'], return_counts = True)[0]) == 1:
                book = np.unique(ssub_df['예매/취소구분'], return_counts = True)[1][0]
                aug_df.loc[np.logical_and(aug_df['공연코드'] == j, aug_df['공연일시'] == d),'좌석점유율'] = book/117
                print(book/max_sit)

            elif len(np.unique(ssub_df['예매/취소구분'], return_counts = True)[0]) == 2:
                book, cancel = np.unique(ssub_df['예매/취소구분'], return_counts = True)[1]
                aug_df.loc[np.logical_and(aug_df['공연코드'] == j, aug_df['공연일시'] == d),'좌석점유율'] = (book-cancel)/117
                print((book-cancel)/max_sit)

        elif len(np.unique(ssub_df['예매/취소구분'], return_counts = True)[0]) == 1:
            book = np.unique(ssub_df['예매/취소구분'], return_counts = True)[1][0]
            aug_df.loc[np.logical_and(aug_df['공연코드'] == j, aug_df['공연일시'] == d),'좌석점유율'] = book/saled
            print(book/saled)
        elif len(np.unique(ssub_df['예매/취소구분'], return_counts = True)[0]) == 2:
            book, cancel = np.unique(ssub_df['예매/취소구분'], return_counts = True)[1]
            aug_df.loc[np.logical_and(aug_df['공연코드'] == j, aug_df['공연일시'] == d),'좌석점유율'] = (book-cancel)/saled
            print((book-cancel)/saled)
        else:
            print('error')

0.625
0.9125
0.9666666666666667
0.14112903225806453
0.08870967741935484
0.08064516129032258
0.40725806451612906
0.4112903225806452
0.4153225806451613
0.40725806451612906
0.4112903225806452


In [ ]:
aug_df.drop(['전송사업자명','좌석수','공연일시','예매/취소매수','예매/취소구분','판매좌석수','예매/취소금액','장당금액','원작자명',
         '극작가명','좌석등급','판매좌석수','공연시작시간','수상실적'],axis = 1, inplace = True)

In [ ]:
aug_df.drop_duplicates(['공연코드','공연시작일','공연시작시분'], inplace = True)

In [ ]:
real_aug = pd.concat([real_aug, aug_df], axis = 0)

In [ ]:
real_aug.to_csv('..\preprocessed_data\예술의 전당\대구콘서트하우스.csv')